Consider again the time series in task 1. Similarly to the first task we want to approximate the maps. 
However, in this task, we are interested in future (or out of samples) predictions of the fluid and solid temperature. 
In other words, the training data consists of time measurements ti registered in a frame [0,T], and we want to forecast the fluid and solid temperature in the frame [T, T_end]
Train a suitable learning model to solve the task based on the data given in Task3

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pandas as pd
from sklearn.model_selection import KFold
mpl.rcParams['figure.dpi']= 300 


Bad key "text.kerning_factor" on line 4 in
/Users/chenyilu/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
torch.manual_seed(42)
np.random.seed(42)

In [6]:
data_training = pd.read_csv("TrainingData.txt", sep =",")

In [7]:
print(data_training)

             t         tf0         ts0
0         0.00  873.000000  293.000000
1      2478.06  873.000000  655.920736
2      4956.12  873.000000  792.106226
3      7434.18  873.000000  842.176577
4      9912.24  873.000000  861.040655
..         ...         ...         ...
205  508002.30  730.794575  730.514961
206  510480.36  730.053663  729.797475
207  512958.42  729.371039  729.133162
208  515436.48  728.734400  728.511278
209  517914.54  728.135229  727.924326

[210 rows x 3 columns]


In [9]:
t_train = data_training["t"]
tf0_train = data_training["tf0"]
ts0_train = data_training["ts0"]

In [10]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs




In [11]:
import numpy as np
import torch

np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')
torch.save(data, open('traindata.pt', 'wb'))

In [14]:
data.shape

(100, 1000)

In [24]:

#parser = argparse.ArgumentParser()
#parser.add_argument('--steps', type=int, default=15, help='steps to run')
#opt = parser.parse_args()
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data_training = pd.read_csv('TrainingData.txt',sep=',')
data_training = torch.from_numpy(data_training.values).float()
data_training = data_training.transpose
data_testing = pd.read_csv('TestingData.txt',sep=',')
data_testing = torch.from_numpy(data_testing.values).float()
data_testing = data_testing.transpose

In [30]:
data_training = pd.read_csv('TrainingData.txt',sep=',')
data_training = transpose(data_training)
type(data_training)

method

In [ ]:
input = torch.from_numpy(data_training[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
for i in range(opt.steps):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
        # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
    plt.close()